# Sequential model using Keras library



Update from the sequential model used in previous notebook with learnings from here: https://www.kaggle.com/cbryant/keras-cnn-statoil-iceberg-lb-0-1995-now-0-1516


In [1]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense, Lambda, Conv2D, MaxPooling2D, BatchNormalization
from keras import applications
import os.path
import pandas as pd
import random
from keras.utils import to_categorical
%matplotlib inline
import matplotlib.pyplot as plt
import cv2
import ilanutils; reload(ilanutils)
from ilanutils import *

Using Theano backend.
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)


In [2]:
path = '/home/ubuntu/courses/deeplearning1/nbs/data/statoil/'
#path = '/Users/ilanrotenberg/projects/courses/deeplearning1/nbs/data/statoil/'

In [3]:
train_path = path + '/train_new.json'
validate_path = path + '/validate.json'
test_path = path + '/test.json'
img_width, img_height = 75, 75
train_batch = pd.read_json(train_path)
validate_batch = pd.read_json(validate_path)
test_batch = pd.read_json(test_path)

img_width, img_height = 75, 75

In [69]:
train_batch.iterrows()

<generator object iterrows at 0x7f5156508690>

In [4]:
train_batch.inc_angle = train_batch.inc_angle.apply(lambda x:np.nan if x =='na' else x)

In [5]:
validate_batch.inc_angle = validate_batch.inc_angle.apply(lambda x:np.nan if x =='na' else x)

In [132]:
test_batch[:5]

,band_1,band_2,id,inc_angle
0,"[-15.863251, -15.201077, -17.887735, -19.17248...","[-21.629612, -21.142353, -23.908337, -28.34524...",5941774d,34.966400
1,"[-26.0589694977, -26.0589694977, -26.058969497...","[-25.7542076111, -25.7542076111, -25.754207611...",4023181e,32.615072
2,"[-14.1410999298, -15.0642414093, -17.375520706...","[-14.745639801, -14.5904102325, -14.3626976013...",b20200e4,37.505433
3,"[-12.167478, -13.706167, -16.54837, -13.572674...","[-24.32222, -26.375538, -24.096739, -23.8769, ...",e7f018bb,34.473900
4,"[-23.3745937347, -26.0271816254, -28.121963501...","[-25.7223434448, -27.0115776062, -23.149162292...",4371c8c3,43.918874


In [6]:
def get_scaled_images(batch):
    
        band_1 = np.array([np.array(band).astype('float32').reshape(img_width,img_height) for band in batch.band_1])
        band_2 = np.array([np.array(band).astype('float32').reshape(img_width,img_height) for band in batch.band_2])
        band_3 = band_1 + band_2
        
        # Rescale
        a = (band_1 - band_1.mean())/(band_1.max() - band_1.min())
        b = (band_2 - band_2.mean())/(band_2.max() - band_2.min())
        c = (band_3 - band_3.mean())/(band_3.max() - band_3.min())
        
        img = np.concatenate([a[:,:,:,np.newaxis],
                              b[:,:,:,np.newaxis],
                              c[:,:,:,np.newaxis]], axis = -1)
        
        return img

In [7]:
X_train = get_scaled_images(train_batch)
X_validate = get_scaled_images(validate_batch)
X_test = get_scaled_images(test_batch)

In [8]:
def get_more_images(batch):
    more_images = []
    vert_images = []
    hor_images = []
    
    for i in range(0,batch.shape[0]):
        a = batch[i,:,:,0]
        b = batch[i,:,:,1]
        c = batch[i,:,:,2]
        
        av = cv2.flip(a,1)
        ah = cv2.flip(a,0)
        bv = cv2.flip(b,1)
        bh = cv2.flip(b,0)
        cv = cv2.flip(c,1)
        ch = cv2.flip(c,0)
        
        vert_images.append(np.dstack((av,bv,cv)))
        hor_images.append(np.dstack((ah,bh,ch)))
        
    v = np.array(vert_images)
    h = np.array(hor_images)
    
    more_images = np.concatenate((batch,v,h))
    
    return more_images

In [9]:
y_train = np.array([np.array(iceberg).astype('uint8') for iceberg in train_batch.is_iceberg])

In [10]:
y_validate = np.array([np.array(iceberg).astype('uint8') for iceberg in validate_batch.is_iceberg])

In [11]:
X_train_large = get_more_images(X_train)
y_train_large = np.concatenate((y_train,y_train,y_train))

In [13]:
y_train_large[0:5]

array([0, 0, 0, 0, 1], dtype=uint8)

In [30]:
y_float_train_large = y_train_large.astype('float')

In [31]:
y_float_train_large[0:5]

array([ 0.,  0.,  0.,  0.,  1.])

In [85]:
y_validate = one_hot(y_validate)
y_train_large = one_hot(y_train_large)

In [14]:
X_train_large = np.moveaxis(X_train_large,3,1)
X_validate = np.moveaxis(X_validate,3,1)
X_test = np.moveaxis(X_test,3,1)

#### Keeping the following model - ended up with 85% accuracy with this model in previous attempt

#### Skip to next model

In [15]:
def get_model():
    model = Sequential([
        Lambda(norm_input, input_shape = (3,75,75)),
        Conv2D(32, (3,3) ,activation='relu'),
        BatchNormalization(axis=1),
        #Dropout(0.1),
        
        Conv2D(32, (3,3),activation='relu'),
        MaxPooling2D(),
        Conv2D(64, (3,3),activation='relu'),
        BatchNormalization(axis=1),
        Dropout(0.1),
        
        Conv2D(64, (3,3),activation='relu'),
        MaxPooling2D(),
        Conv2D(128, (3,3),activation='relu'),
        BatchNormalization(axis=1),
        #Dropout(0.1),
        
        Conv2D(128, (3,3),activation='relu'),
        MaxPooling2D(),
        
        Flatten(),
        BatchNormalization(),
        Dense(512, activation ='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(128, activation ='relu'),
        BatchNormalization(),
        Dense(1, activation='softmax')
    ])
    model.compile(optimizer = 'Adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

#### This model is structured like the link from the first cell

In [40]:
def get_conv_model():
    model = Sequential([
        Conv2D(64, kernel_size = (3,3), activation='relu', input_shape = (3, 75,75)),
        MaxPooling2D(pool_size = (3,3), strides = (2,2)),
        BatchNormalization(axis=1),
        Dropout(0.2),
        
        Conv2D(128, kernel_size = (3,3), activation='relu'),
        MaxPooling2D(pool_size = (2,2), strides = (2,2)),
        BatchNormalization(axis=1),
        Dropout(0.2),
        
        Conv2D(128, kernel_size = (3,3), activation='relu'),
        MaxPooling2D(pool_size = (2,2), strides = (2,2)),
        BatchNormalization(axis=1),
        Dropout(0.2),
        
        Conv2D(64, kernel_size = (3,3), activation='relu'),
        MaxPooling2D(pool_size = (2,2), strides = (2,2)),
        BatchNormalization(axis=1),
        #Dropout(0.1),
        
        Flatten(),
        
        Dense(512, activation ='relu'),
        BatchNormalization(),
        Dropout(0.6),
        Dense(256, activation ='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer = 'Adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [41]:
model = get_conv_model()

In [42]:
model.optimizer.lr = 0.001
model.fit(X_train_large, y_float_train_large, batch_size = 64, epochs = 10, verbose = 1,
                       validation_data = (X_validate, y_validate))

Train on 3369 samples, validate on 481 samples
Epoch 1/10
3369/3369 [==============================] - 8s 2ms/step - loss: 0.5404 - acc: 0.7355 - val_loss: 0.7832 - val_acc: 0.4865
Epoch 2/10
3369/3369 [==============================] - 8s 2ms/step - loss: 0.4057 - acc: 0.8160 - val_loss: 0.7781 - val_acc: 0.4865
Epoch 3/10
3369/3369 [==============================] - 8s 2ms/step - loss: 0.3450 - acc: 0.8468 - val_loss: 1.0377 - val_acc: 0.4865
Epoch 4/10
3369/3369 [==============================] - 8s 2ms/step - loss: 0.2975 - acc: 0.8679 - val_loss: 0.8064 - val_acc: 0.4886
Epoch 5/10
3369/3369 [==============================] - 8s 2ms/step - loss: 0.2766 - acc: 0.8765 - val_loss: 1.0820 - val_acc: 0.4865
Epoch 6/10
3369/3369 [==============================] - 8s 2ms/step - loss: 0.2580 - acc: 0.8866 - val_loss: 1.0926 - val_acc: 0.5281
Epoch 7/10
3369/3369 [==============================] - 8s 2ms/step - loss: 0.2316 - acc: 0.9020 - val_loss: 1.0432 - val_acc: 0.6154
Epoch 8/10
3369

In [45]:
model.optimizer.lr = 0.001
model.fit(X_train_large, y_float_train_large, batch_size = 64, epochs = 1, verbose = 1,
                       validation_data = (X_validate, y_validate))

Train on 3369 samples, validate on 481 samples
Epoch 1/1
3369/3369 [==============================] - 8s 2ms/step - loss: 0.0068 - acc: 0.9979 - val_loss: 0.5077 - val_acc: 0.9002


## Ensemble model

~~Todo: use learnings from previous section to update the ensemble model to generate better predictions~~ done 14 December

In [110]:
def get_ensemble() :
    model = get_conv_model()
    #model.optimizer.lr = 0.00001
    model.fit(X_train_large, y_train_large, batch_size = 64, epochs = 4, verbose = 0,
                       validation_data = (X_validate, y_validate))
    model.optimizer.lr = 0.001
    model.fit(X_train_large, y_train_large, batch_size = 64, epochs = 20, verbose = 1,
                       validation_data = (X_validate, y_validate))
    return model

In [111]:
models = [get_ensemble() for i in range(3)]

Train on 3369 samples, validate on 481 samples
Epoch 1/30
3369/3369 [==============================] - 6s 2ms/step - loss: 0.2724 - acc: 0.8813 - val_loss: 0.3170 - val_acc: 0.8711
Epoch 2/30
3369/3369 [==============================] - 6s 2ms/step - loss: 0.2739 - acc: 0.8828 - val_loss: 0.4322 - val_acc: 0.8212
Epoch 3/30
3369/3369 [==============================] - 6s 2ms/step - loss: 0.2477 - acc: 0.8994 - val_loss: 0.4656 - val_acc: 0.8337
Epoch 4/30
3369/3369 [==============================] - 6s 2ms/step - loss: 0.2378 - acc: 0.9015 - val_loss: 0.3235 - val_acc: 0.8669
Epoch 5/30
3369/3369 [==============================] - 6s 2ms/step - loss: 0.2040 - acc: 0.9160 - val_loss: 0.2548 - val_acc: 0.9044
Epoch 6/30
3369/3369 [==============================] - 6s 2ms/step - loss: 0.2079 - acc: 0.9115 - val_loss: 0.2452 - val_acc: 0.9023
Epoch 7/30
3369/3369 [==============================] - 6s 2ms/step - loss: 0.1851 - acc: 0.9243 - val_loss: 0.2710 - val_acc: 0.9106
Epoch 8/30
3369

In [46]:
model_path = path+'models/'

In [47]:
for i,m in enumerate(models):
    m.save_weights(model_path + 'cnn_statoil_good_model_02_jan' + str(i) + '.pkl')

TypeError: 'Sequential' object is not iterable

In [49]:
model.save_weights(model_path + 'cnn_statoil_good_model_02_jan_1.pkl')

In [114]:
evals = np.array([m.evaluate(X_validate, y_validate, batch_size=256) for m in models])

481/481 [==============================] - 0s 607us/step


In [115]:
evals.mean(axis=0)

array([ 0.50136084,  0.9009009 ])

In [119]:
all_preds = np.stack([m.predict(X_test, batch_size = 256) for m in models])

In [120]:
all_preds.shape

(3, 8424, 2)

In [121]:
avg_preds = all_preds.mean(axis=0)

In [122]:
avg_preds[:5]

array([[  9.99989033e-01,   1.09978355e-05],
       [  2.63524451e-03,   9.97364759e-01],
       [  8.38365853e-01,   1.61634162e-01],
       [  1.58535764e-01,   8.41464221e-01],
       [  2.99966410e-02,   9.70003366e-01]], dtype=float32)

In [123]:
avg_preds[:,1]

array([  1.09978355e-05,   9.97364759e-01,   1.61634162e-01, ...,
         1.43135694e-04,   7.58578837e-01,   9.99999821e-01], dtype=float32)

In [124]:
evals.shape

(3, 2)

In [125]:
evals

array([[ 0.41178337,  0.90644489],
       [ 0.54441499,  0.91268191],
       [ 0.54788417,  0.88357589]])

In [50]:
pred_single = model.predict(X_test,batch_size = 256)

In [51]:
pred_single[0:5]

array([[  4.77482843e-07],
       [  6.92913651e-01],
       [  9.99999642e-01],
       [  9.99980569e-01],
       [  9.28289711e-01]], dtype=float32)

In [135]:
preds = model.predict(X_test)

In [57]:
submission = pd.DataFrame()
submission['id']=test_batch['id']
submission['is_iceberg']=pred_clipped[:]
submission.to_csv(path+'submission_02_jan_1.csv', index = False)

In [58]:
submission.head(10)

,id,is_iceberg
0,5941774d,0.010000
1,4023181e,0.692914
2,b20200e4,0.990000
3,e7f018bb,0.990000
4,4371c8c3,0.928290
5,a8d9b1fd,0.990000
6,29e7727e,0.010000
7,92a51ffb,0.990000
8,c769ac97,0.976223
9,aee0547d,0.010000


In [56]:
pred_clipped = pred_single[:].clip(0.01, 0.99)

In [59]:
from IPython.display import FileLink, FileLinks
import os, sys
submit_path = os.getcwd()
submit_path

'/home/ubuntu/courses/deeplearning1/nbs/statoil-nb'

In [60]:
%cd $path

/home/ubuntu/courses/deeplearning1/nbs/data/statoil


In [206]:
FileLink('submission171214.csv')

/home/ubuntu/courses/deeplearning1/nbs/data/statoil/submission171214.csv

In [61]:
FileLinks('.')

./
  test.json
  submission_02_jan_1.csv
  sample_submission.csv.7z
  validate.json
  train.json
  submission171214.csv
  submission.csv
  submission17124.csv
  submission_14_december.csv
  submission1.csv
  submission_02_jan.csv
  train_new.json
./data/processed/
  sample_submission.csv
./models/
  cnn_statoil_14december0.pkl
  cnn_statoil_14december1.pkl
  cnn_statoil_good_model_14_dec2.pkl
  cnn_statoil_2.pkl
  cnn_statoil_good_model_14_dec1.pkl
  cnn_statoil_0.pkl
  cnn_statoil_1.pkl
  cnn_statoil_good_model_14_dec0.pkl
  cnn_statoil_good_model_02_jan_1.pkl
  cnn_statoil_14december2.pkl
./models/train_data.bc/
  __attrs__
./models/train_data.bc/data/
  __1016.blp
  __282.blp
  __158.blp
  __615.blp
  __984.blp
  __1060.blp
  __148.blp
  __87.blp
  __352.blp
  __960.blp
  __668.blp
  __526.blp
  __713.blp
  __191.blp
  __522.blp
  __852.blp
  __976.blp
  __402.blp
  __446.blp
  __358.blp
  __703.blp
  __135.blp
  __558.blp
  __616.blp
  __514.blp
  __149.blp
  __4.blp
  __601.blp
  __947.blp
  __867.blp
  __202.blp
  __341.blp
  __596.blp
  __646.blp
  __385.blp
  __771.blp
  __276.blp
  __188.blp
  __697.blp
  __80.blp
  __539.blp
  __485.blp
  __775.blp
  __1036.blp
  __11.blp
  __1027.blp
  __856.blp
  __923.blp
  __698.blp
  __939.blp
  __174.blp
  __888.blp
  __209.blp
  __989.blp
  __968.blp
  __140.blp
  __365.blp
  __360.blp
  __568.blp
  __878.blp
  __686.blp
  __215.blp
  __815.blp
  __77.blp
  __629.blp
  __672.blp
  __300.blp
  __309.blp
  __1054.blp
  __431.blp
  __889.blp
  __411.blp
  __278.blp
  __271.blp
  __801.blp
  __610.blp
  __739.blp
  __936.blp
  __441.blp
  __1052.blp
  __242.blp
  __88.blp
  __1006.blp
  __480.blp
  __756.blp
  __930.blp
  __804.blp
  __622.blp
  __750.blp
  __410.blp
  __594.blp
  __251.blp
  __206.blp
  __1031.blp
  __635.blp
  __92.blp
  __881.blp
  __471.blp
  __275.blp
  __207.blp
  __988.blp
  __413.blp
  __662.blp
  __611.blp
  __826.blp
  __426.blp
  __959.blp
  __836.blp
  __753.blp
  __5.blp
  __406.blp
  __28.blp
  __461.blp
  __659.blp
  __49.blp
  __235.blp
  __1002.blp
  __675.blp
  __286.blp
  __536.blp
  __417.blp
  __247.blp
  __533.blp
  __995.blp
  __326.blp
  __548.blp
  __117.blp
  __1008.blp
  __769.blp
  __1010.blp
  __105.blp
  __234.blp
  __975.blp
  __29.blp
  __709.blp
  __52.blp
  __1047.blp
  __155.blp
  __452.blp
  __793.blp
  __419.blp
  __903.blp
  __642.blp
  __167.blp
  __773.blp
  __1035.blp
  __364.blp
  __685.blp
  __717.blp
  __588.blp
  __103.blp
  __298.blp
  __974.blp
  __1053.blp
  __971.blp
  __570.blp
  __475.blp
  __430.blp
  __196.blp
  __190.blp
  __796.blp
  __48.blp
  __1058.blp
  __499.blp
  __302.blp
  __650.blp
  __1004.blp
  __638.blp
  __249.blp
  __684.blp
  __361.blp
  __50.blp
  __572.blp
  __82.blp
  __468.blp
  __911.blp
  __32.blp
  __459.blp
  __416.blp
  __376.blp
  __719.blp
  __375.blp
  __512.blp
  __747.blp
  __925.blp
  __605.blp
  __71.blp
  __655.blp
  __842.blp
  __754.blp
  __387.blp
  __899.blp
  __19.blp
  __952.blp
  __932.blp
  __383.blp
  __146.blp
  __941.blp
  __1005.blp
  __1029.blp
  __220.blp
  __1011.blp
  __328.blp
  __755.blp
  __96.blp
  __263.blp
  __490.blp
  __895.blp
  __523.blp
  __335.blp
  __156.blp
  __628.blp
  __322.blp
  __231.blp
  __311.blp
  __194.blp
  __1013.blp
  __585.blp
  __62.blp
  __1039.blp
  __639.blp
  __63.blp
  __184.blp
  __120.blp
  __450.blp
  __789.blp
  __811.blp
  __958.blp
  __421.blp
  __40.blp
  __532.blp
  __716.blp
  __849.blp
  __1034.blp
  __599.blp
  __507.blp
  __880.blp
  __873.blp
  __8.blp
  __934.blp
  __211.blp
  __229.blp
  __1024.blp
  __391.blp
  __491.blp
  __78.blp
  __313.blp
  __354.blp
  __818.blp
  __492.blp
  __805.blp
  __192.blp
  __267.blp
  __768.blp
  __325.blp
  __517.blp
  __134.blp
  __665.blp
  __345.blp
  __931.blp
  __374.blp
  __498.blp
  __843.blp
  __201.blp
  __950.blp
  __733.blp
  __76.blp
  __577.blp
  __65.blp
  __677.blp
  __933.blp
  __147.blp
  __788.blp
  __546.blp
  __566.blp
  __198.blp
  __583.blp
  __538.blp
  __904

http://ec2-52-43-137-167.us-west-2.compute.amazonaws.com:8888/notebooks/statoil-nb/sample_submission.csv.7z


http://ec2-52-43-137-167.us-west-2.compute.amazonaws.com:8888/notebooks/statoil-nb/submission_02_jan_1.csv